# QCUT05 : Explore hologram data  : Impact of Weather conditions

- author Sylvie Dagoret-Campagne
- creation date 2026-01-21 : version run2026_v01
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.qualitycuts import scatter_datetime
from mysitcom.auxtel.qualitycuts import strip_datetime
from mysitcom.auxtel.qualitycuts import bar_counts_by_night
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.qualitycuts import stripplot_target_vs_time
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.qualitycuts import summarize_dccd_chi2
from mysitcom.auxtel.qualitycuts import plot_param_histogram_grid
from mysitcom.auxtel.qualitycuts import plot_params_and_chi2_vs_time
from mysitcom.auxtel.qualitycuts import plot_param_chi2_correlation_grid
from mysitcom.auxtel.qualitycuts import plot_param2_vs_param1_colored_by_time
from mysitcom.auxtel.qualitycuts import plot_param_difference_vs_time
from mysitcom.auxtel.qualitycuts import plot_param_difference_vs_time_colored_by_chi2
from mysitcom.auxtel.qualitycuts import plot_single_param_vs_time_colored_by_chi2

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_QCUT05"
prefix = "qcut05"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from QCUT00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `QCUT00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec[["id","FILTER"]]

In [ ]:
df_spec["FILTER"].unique()

## Correct the time units

In [ ]:
#df_spec["DATE-OBS"] = (
#    pd.to_datetime(df_spec["DATE-OBS"], utc=True, errors="coerce")
#    .dt.tz_convert(None)
#)

## Angle uniformization

In [ ]:
#df["angle_180"] = ((df["angle_360"] + 180) % 360) - 180

In [ ]:
flag_angles_m180_p180 = False

In [ ]:
if not flag_angles_m180_p180:
    df_spec["DOMEAZ"] = ((df_spec["DOMEAZ"] + 180) % 360) - 180
    df_spec["RA"] = ((df_spec["RA"] + 180) % 360) - 180
    df_spec["WINDDIR"] = ((df_spec["WINDDIR"] + 180) % 360) - 180
    flag_angles_m180_p180 = True

## Histograms of parameters

In [ ]:
params = [
    'DOMEAZ', 'AZ', 'EL', 'RA','AIRMASS',
    "WINDSPD",
    "WINDDIR", 
    "PARANGLE", 
    "OUTTEMP",
    "OUTPRESS",
    "OUTHUM",
]


filter_order = ["empty", "BG40_65mm_1", "OG550_65mm_1"]

param_ranges = {
    # mount motion image degradation
    "mount_motion_image_degradation_x": (0.0, 0.5),
    "mount_motion_image_degradation_y": (0.0, 0.5),
    "mount_motion_image_degradation_az_x": (0.0, 0.5),
    "mount_motion_image_degradation_az_y": (0.0, 0.5),
    "mount_motion_image_degradation_el_x": (0.0, 0.5),
    "mount_motion_image_degradation_el_y": (0.0, 0.5),

    # mount jitter rms
    "mount_jitter_rms_x": (0.0, 0.5),
    "mount_jitter_rms_y": (0.0, 0.5),
    "mount_jitter_rms_az_x": (0.0, 0.5),
    "mount_jitter_rms_az_y": (0.0, 0.5),
    "mount_jitter_rms_el_x": (0.0, 0.5),
    "mount_jitter_rms_el_y": (0.0, 0.5),
    "mount_jitter_rms_rot_x": (0.0, 0.5),
    "mount_jitter_rms_rot_y": (0.0, 0.5),

    # seeing & focus
    "dimm_seeing_x": (0.0, 2.5),
    "dimm_seeing_y": (0.0, 2.5),
    "focus_z_x": (0.0, 0.15),
    "focus_z_y": (0.0, 0.15),
}



fig, axs = plot_param_histogram_grid(
    df=df_spec,
    params=params,
    filter_col="FILTER",
    filter_order=filter_order,
    param_ranges=param_ranges,
    bins=40,
    stacked=True,
    logy=False,
)


### Convert DATE-OBS to pd_to_datetime

In [ ]:
df_spec["DATE-OBS"].dtype

In [ ]:
df_spec["DATE-OBS"] = pd.to_datetime(
    df_spec["DATE-OBS"],
    utc=True,
    errors="coerce").dt.tz_convert(None)

## Plot params vs time

In [ ]:
params = [
    'DOMEAZ', 'AZ', 'EL', 'RA','AIRMASS',
    "WINDSPD",
    "WINDDIR", 
    "PARANGLE", 
    "OUTTEMP",
    "OUTPRESS",
    "OUTHUM",
]




params_few = [
    "WINDSPD",
    "WINDDIR", 
]
param_colors = {
    "WINDSPD": "tab:purple",
    "WINDDIR": "tab:pink",
}


fig, axs = plot_params_and_chi2_vs_time(
    df=df_spec,
    time_col="DATE-OBS",
    filter_col="FILTER",
    chi2_col="CHI2_FIT",
    params=params,
#    param_ylim={
#        "D_CCD": (-0.5, 0.5),
#        "FOCUS_OFFSET": (-200, 200),
#    },
    chi2_cut=CHI2CUT,
    suptitle="weather parameters and CHI2 vs time",
)
plt.show()

## Plot correlation functions

In [ ]:
params = [
    'DOMEAZ', 'AZ', 'EL', 'RA','AIRMASS',
    "WINDSPD",
    "WINDDIR", 
    "PARANGLE", 
    "OUTTEMP",
    "OUTPRESS",
    "OUTHUM",
]




filter_order = ["empty", "BG40_65mm_1", "OG550_65mm_1"]

fig, axs = plot_param_chi2_correlation_grid(
    df=df_spec,
    params=params,
    chi2_col="CHI2_FIT",
    filter_col="FILTER",
    filter_order=filter_order,
    param_ranges=param_ranges,
    chi2_range=(1., 5000.),
)
plt.show()

## Check azimuth

In [ ]:
plot_param2_vs_param1_colored_by_time(
    df=df_spec,
    time_col="DATE-OBS",
    param1="OUTPRESS",
    param2="OUTTEMP",
    cmap="jet",
    aspect="auto"
)
plt.show()

In [ ]:
#plot_param_difference_vs_time(
#    df=df_spec,
#    time_col="DATE-OBS",
#    param1="DOMEAZ",
#    param2="AZ",
#    zoomdiff = (-50.,50.)
#)

In [ ]:
plot_single_param_vs_time_colored_by_chi2(
    df=df_spec,
    time_col="DATE-OBS",
    param="OUTPRESS",
    chi2_col="CHI2_FIT",
    chi2_range=(1, 1e4),
    cmap="jet",
#    zoomdiff = (-50.,50.)
)
plt.show()

In [ ]:
plot_single_param_vs_time_colored_by_chi2(
    df=df_spec,
    time_col="DATE-OBS",
    param="OUTTEMP",
    chi2_col="CHI2_FIT",
    chi2_range=(1, 1e4),
    cmap="jet",
#    zoomdiff = (-50.,50.)
)
plt.show()

In [ ]:
plot_single_param_vs_time_colored_by_chi2(
    df=df_spec,
    time_col="DATE-OBS",
    param="OUTHUM",
    chi2_col="CHI2_FIT",
    chi2_range=(1, 1e4),
    cmap="jet",
#    zoomdiff = (-50.,50.)
)
plt.show()